In [1]:
# Goal: pull stock data for those companies listed in the S&P500

In [14]:
# import packages:

import pandas as pd
import numpy as np
import os
import time as tm
import pandas_datareader as pdr
import datetime
import requests_cache
import random
import glob

In [ ]:
# research articles for the topic:

# best moving average crosses: https://www.yahoo.com/entertainment/s/study-determines-best-moving-average-195042216.html


In [53]:
# functions:

def pull_stocks(stock_list, start_date, end_date, file_save_location, sleep_min = 2, sleep_max = 10, expire_days = 3):
    
    session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_days)
    
    for stock in stock_list:
        
        start = tm.time()
        print('retrieving: ' , stock)
    
        ## pull and data generation:
        df = pdr.get_data_yahoo(stock.strip(), start = start_date, end = end_date, session = session)
        df['ticker'] = stock
        df = df.reset_index()
        df.to_csv(os.path.join(file_save_location, stock.strip() +'.csv'), index = False)
        print('saving: ', stock, ' data to: ', os.path.join(file_save_location, stock.strip() +'.csv'))

        end = tm.time()
        total = (end - start) / 60
        print(f"completed retrieving: {stock.strip()} data in: {total}")
        
        sleep_time = random.randint(2, 10)
        print(f"sleeping for: {sleep_time} seconds")
       

        # sleep between pulls so to not arouse suspicion:
        tm.sleep(sleep_time)

        del df, start, end, total, sleep_time
        

def combine_historic_data(data_location, output_location):
    
    counter = 0

    for file in glob.glob(file_path):

        print(f"reading {file} from directory")

        if counter == 0:
            df = pd.read_csv(file)
            df.to_csv(new_file_path, index = False, mode = 'w', header = True)
            del df
            print(f"wrote {file} to new file: {new_file_path}")

        else:
            df = pd.read_csv(file)
            df = df.reset_index(drop = True)
            df.to_csv(new_file_path, index = False, mode = 'a', header = False)
            del df
            print(f"wrote {file} to existing file: {new_file_path}")

        # fix odd index issue:
        df = pd.read_csv(new_file_path)
        cols = [c for c in df.columns if c.lower() != 'Unnamed']
        df = df[cols]
        df.to_csv(new_file_path, index = False)

        counter += 1
        

def fix_columns(dataframe):
    
    '''function that takes a list of columns and modifies them to be easier to read -- assign to df.columns'''
    
    column_string_replace = ['\n','@',' ','__', '/', '-']

    columns = dataframe.columns

    columns = columns.map(lambda x: x.strip())
    columns = columns.map(lambda x : x.lower())

    for string in column_string_replace:
        columns = columns.map(lambda x : x.replace(string, '_') if isinstance (x, (str, bytes)) else x)

    return columns

In [3]:
# read in the S&P 500 stock list:

# No longer using this, sticking with sector spyders : sp_stox = pd.read_csv('../01_data/01_raw/test_tickers.csv')


In [20]:
# test the function:

start = '2018-01-01'
end = '2020-12-30'
file_location = '../01_data/01_raw/historic_data'

etfs = ['XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU', 'SPY']

pull_stocks(stock_list = etfs, start_date = start, end_date = end, file_save_location = file_location)

retrieving:  XLC
saving:  XLC  data to:  ../01_data/01_raw/historic_data\XLC.csv
completed retrieving: XLC data in: 0.04297378857930501
sleeping for: 10 seconds
retrieving:  XLY
saving:  XLY  data to:  ../01_data/01_raw/historic_data\XLY.csv
completed retrieving: XLY data in: 0.024610412120819092
sleeping for: 4 seconds
retrieving:  XLP
saving:  XLP  data to:  ../01_data/01_raw/historic_data\XLP.csv
completed retrieving: XLP data in: 0.03082280953725179
sleeping for: 2 seconds
retrieving:  XLE
saving:  XLE  data to:  ../01_data/01_raw/historic_data\XLE.csv
completed retrieving: XLE data in: 0.0250296155611674
sleeping for: 10 seconds
retrieving:  XLF
saving:  XLF  data to:  ../01_data/01_raw/historic_data\XLF.csv
completed retrieving: XLF data in: 0.023457419872283936
sleeping for: 5 seconds
retrieving:  XLV
saving:  XLV  data to:  ../01_data/01_raw/historic_data\XLV.csv
completed retrieving: XLV data in: 0.02597554922103882
sleeping for: 3 seconds
retrieving:  XLI
saving:  XLI  data t

In [54]:
# Combine historic data:

file_path = '../01_data/01_raw/historic_data/*.csv'
new_file_path = '../01_data/01_raw/combined_data.csv'

combine_historic_data(data_location = file_path, output_location = new_file_path)

reading ../01_data/01_raw/historic_data\SPY.csv from directory
wrote ../01_data/01_raw/historic_data\SPY.csv to new file: ../01_data/01_raw/combined_data.csv
reading ../01_data/01_raw/historic_data\TEST.csv from directory
wrote ../01_data/01_raw/historic_data\TEST.csv to existing file: ../01_data/01_raw/combined_data.csv
reading ../01_data/01_raw/historic_data\XLB.csv from directory
wrote ../01_data/01_raw/historic_data\XLB.csv to existing file: ../01_data/01_raw/combined_data.csv
reading ../01_data/01_raw/historic_data\XLC.csv from directory
wrote ../01_data/01_raw/historic_data\XLC.csv to existing file: ../01_data/01_raw/combined_data.csv
reading ../01_data/01_raw/historic_data\XLE.csv from directory
wrote ../01_data/01_raw/historic_data\XLE.csv to existing file: ../01_data/01_raw/combined_data.csv
reading ../01_data/01_raw/historic_data\XLF.csv from directory
wrote ../01_data/01_raw/historic_data\XLF.csv to existing file: ../01_data/01_raw/combined_data.csv
reading ../01_data/01_raw

In [81]:
# test shifts:

df_test = pd.read_csv('../01_data/01_raw/historic_data/XLU.csv')

# have the full dataframe to translate:

df = pd.read_csv('../01_data/01_raw/combined_data.csv')

In [93]:
# engineer features one at a time and then we can engineer for the broader dataframe:

# sort dataframe by date and ticker:

# next trading day of week:

# create target variable with shift (using actual closing price since we want to predict higher or lower):

df_test['target'] = df_test['Close'].shift(periods = -1)

df['target_variable'] = df.groupby(by = ['ticker'])['Close'].shift(periods = -1).reset_index(drop = True)

# features:

# 7, 14, 20, 30, 50 SMA:

df_test['7_price_sma'] = df_test['Close'].rolling(7).mean().reset_index(drop = True)

df['7_price_sma'] = df.groupby(by = 'ticker').rolling(7)['Close'].mean().reset_index(drop=True)
df['14_price_sma'] = df.groupby(by = 'ticker').rolling(14)['Close'].mean().reset_index(drop=True)
df['21_price_sma'] = df.groupby(by = 'ticker').rolling(21)['Close'].mean().reset_index(drop=True)
df['30_price_sma'] = df.groupby(by = 'ticker').rolling(30)['Close'].mean().reset_index(drop=True)
df['50_price_sma'] = df.groupby(by = 'ticker').rolling(50)['Close'].mean().reset_index(drop=True)

# rolling standard deviation for the same features:

df_test['7_std'] = df_test['Close'].rolling(7).std().reset_index(drop = True)

df['7_price_std'] = df.groupby(by = 'ticker').rolling(7)['Close'].std().reset_index(drop=True)
df['14_price_std'] = df.groupby(by = 'ticker').rolling(14)['Close'].std().reset_index(drop=True)
df['21_price_std'] = df.groupby(by = 'ticker').rolling(21)['Close'].std().reset_index(drop=True)
df['30_price_std'] = df.groupby(by = 'ticker').rolling(30)['Close'].std().reset_index(drop=True)
df['50_price_std'] = df.groupby(by = 'ticker').rolling(50)['Close'].std().reset_index(drop=True)
# same feautures for the volume:

# standard deviation ratios (e.g., ratio of 20 SMA to 50 SMA of std dev, ratio of std deviation to current price or SMA to current price)

# consecutive days above/below SMA bands and gaps ($ and %):

# create indicator variables for whether price above or below a given SMA:

df_test['price_above_7_sma'] = np.where(df_test['Close'] > df_test['7_price_sma'], 1, 0 )

df_test['count'] = df_test.groupby((df_test['price_above_7_sma'] != df_test['price_above_7_sma'].shift(1)).cumsum()).cumcount()+1
#df['count'] = df.groupby((df['col'] != df['col'].shift(1)).cumsum()).cumcount()+1

df['price_above_7_sma'] = np.where(df['Close'] > df['7_price_sma'], 1, 0 )

df['days_above_7_sma'] = df.groupby(by = ['ticker', (df_test['price_above_7_sma'] != df_test['price_above_7_sma'].shift(1)).cumsum()]).cumcount()+1
df['price_above_14_sma'] = np.where(df['Close'] > df['14_price_sma'], 1, 0 )
df['price_above_21_sma'] = np.where(df['Close'] > df['21_price_sma'], 1, 0 )
df['price_above_30_sma'] = np.where(df['Close'] > df['30_price_sma'], 1, 0 )
df['price_above_50_sma'] = np.where(df['Close'] > df['50_price_sma'], 1, 0 )

# consecutive days above/below SMA bands and gaps :



# exponential moving averages:


# range relative to closing price:

# previous day's range:

# low = high, number of times in past N days

# within 20 day bollineger band, % away from 20-day high bollinger band, % away from 20-day low bollinger band

# Moving average of range (standardized? or indexed):

# ratio of high to low:

# difference/ ratio to 52-week high

# 10, 7, 14, 21 day standard deviation of price

# Valume change over past N periods:

# volume range, indexed to a time period: 

# number of trailing days above N day SMA

# gap to N day SMA (price and volume)

# Interaction of price and volume:

# Ratio of price change % to volume change % over N periods:

# days since last 52-week high:

# Seasonality: (time of year, next trading day DOW, etc.)

# Correlation/movement with ther features:

# 

In [94]:
df_test.tail(20)

,Date,High,Low,Open,Close,Volume,Adj Close,ticker,target,7_sma,7_std,7_price_sma,price_above_7_sma,count
735,2020-12-02,63.730000,62.580002,62.980000,63.700001,12198800.0,63.166183,XLU,63.040001,63.680000,0.533886,63.680000,1,1
736,2020-12-03,63.660000,62.869999,63.599998,63.040001,9376800.0,62.511715,XLU,62.410000,63.607143,0.586764,63.607143,0,1
737,2020-12-04,63.250000,62.060001,63.099998,62.410000,10502500.0,61.886993,XLU,62.759998,63.347143,0.663042,63.347143,0,2
738,2020-12-07,63.070000,62.189999,62.200001,62.759998,8695700.0,62.234058,XLU,62.570000,63.112857,0.498287,63.112857,0,3
739,2020-12-08,62.980000,62.090000,62.349998,62.570000,10502700.0,62.045650,XLU,62.459999,62.942857,0.440330,62.942857,0,4
740,2020-12-09,62.720001,62.040001,62.619999,62.459999,11484900.0,61.936573,XLU,62.139999,62.888571,0.477020,62.888571,0,5
741,2020-12-10,62.630001,61.849998,62.430000,62.139999,10801400.0,61.619255,XLU,62.240002,62.725714,0.514259,62.725714,0,6
742,2020-12-11,62.320000,61.820000,61.880001,62.240002,6675700.0,61.718418,XLU,61.919998,62.517143,0.307935,62.517143,0,7
743,2020-12-14,63.200001,61.860001,62.680000,61.919998,13604400.0,61.401096,XLU,63.139999,62.357142,0.280756,62.357142,0,8
744,2020-12-15,63.299999,61.900002,62.209999,63.139999,11178600.0,62.610874,XLU,62.410000,62.461428,0.409651,62.461428,1,1


In [95]:
df.tail(20)

,Date,High,Low,Open,Close,Volume,Adj Close,ticker,target_variable,7_price_sma,...,14_price_std,21_price_std,30_price_std,50_price_std,price_above_7_sma,price_above_14_sma,price_above_21_sma,price_above_30_sma,price_above_50_sma,days_above_7_sma
9679,2020-12-02,157.809998,156.770004,157.809998,157.440002,2154400.0,157.148712,XLY,157.880005,157.471429,...,2.242395,2.944426,4.189263,4.352677,0,1,1,1,1,8925
9680,2020-12-03,158.750000,157.479996,157.479996,157.880005,3005700.0,157.587906,XLY,158.029999,157.828572,...,1.870725,2.556133,4.297855,4.240667,1,1,1,1,1,8926
9681,2020-12-04,158.429993,157.720001,158.199997,158.029999,2089100.0,157.737625,XLY,157.669998,157.877143,...,1.763393,2.433285,4.401953,4.102514,1,1,1,1,1,8927
9682,2020-12-07,157.750000,156.869995,157.679993,157.669998,4215300.0,157.378296,XLY,157.339996,157.847142,...,1.766180,2.451764,4.487723,4.011104,0,1,1,1,1,8928
9683,2020-12-08,157.440002,156.119995,156.910004,157.339996,2478400.0,157.048904,XLY,157.190002,157.721427,...,1.694395,2.413436,4.517192,3.999270,0,1,1,1,1,8929
9684,2020-12-09,158.449997,156.520004,158.369995,157.190002,6030400.0,156.899185,XLY,157.050003,157.711428,...,1.448757,2.313460,4.526878,3.935574,0,1,1,1,1,8930
9685,2020-12-10,157.720001,155.929993,156.580002,157.050003,2903000.0,156.759445,XLY,156.300003,157.514287,...,1.251310,2.128322,4.288424,3.894847,0,0,1,1,1,8931
9686,2020-12-11,157.029999,155.229996,156.470001,156.300003,2067100.0,156.010834,XLY,156.529999,157.351430,...,0.795562,2.013997,4.048381,3.880645,0,0,1,1,1,8932
9687,2020-12-14,158.619995,156.479996,157.720001,156.529999,3407500.0,156.240402,XLY,158.639999,157.158572,...,0.606612,1.622205,3.458293,3.849146,0,0,1,1,1,8933
9688,2020-12-15,158.669998,157.190002,157.910004,158.639999,2326300.0,158.346497,XLY,159.679993,157.245714,...,0.679081,1.531455,2.877549,3.899915,1,1,1,1,1,8934
